In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.5 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
m = gp.Model("ls")

capacitated = False

Restricted license - for non-production use only - expires 2024-10-28


In [ ]:
# Time (e.g. months)
n = 8
t = range(0, n)

# Costs for production of one product for each month
p = [100 for i in t]

# Costs for storing a product for each month
h = [5 for i in t]

# Costs for armoring the machines for each month (has to be done at most once for each month)
q = [5000 for i in t]

# Demand for products for each month
d = [400, 400, 800, 800, 1200, 1200, 1200, 1200]

# Production Amount for each month
x = m.addVars(n, name="x", vtype=GRB.INTEGER)

# Storage Amount for each month
s = m.addVars(n + 1, name="s", vtype=GRB.INTEGER)

# Production Preparation necessary for each month (0 or 1)
y = m.addVars(n, name="y", vtype=GRB.BINARY)

# Production Capacity for each month (in this case unbounded -> LS-U)
if capacitated:
	M = [7000, 0, 0, 0, 0, 0, 0, 0]
else:
	M = m.addVars(n, name="M", vtype=GRB.INTEGER)

In [ ]:
m.setObjective(gp.quicksum(p[i] * x[i] + q[i] * y[i] + h[i] * s[i+1] for i in t), GRB.MINIMIZE)

In [ ]:
# Stored products from the previous month plus the number of products produced in the current
# month must fulfill the demand while the rest of the products must be stored for the next month
m.addConstrs((s[i-1] + x[i-1] == d[i-1] + s[i] for i in range(1, n+1)), "c1")
m.addConstr((s[0] + x[0] == d[0] + s[1]), "c2")

# The number of products stored in the first month must be equal to 200 (Initial stock)
m.addConstr(s[0] == 200, "c3")

# The number of products stored in the last month must be equal to 0 (Final stock)
m.addConstr(s[n] == 0, "c4")

# If products are being produced in the current month the machines must be prepared
m.addConstrs((x[i] <= M[i]*y[i] for i in t), "c5")

# There cant be a negative number of products stored in the warehouse or produced
m.addConstrs((x[i] >= 0 for i in t), "c6")
m.addConstrs((s[i] >= 0 for i in t), "c7")
m.addConstrs((M[i] >= 0 for i in t), "c8");

In [ ]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 35 rows, 33 columns and 53 nonzeros
Model fingerprint: 0xa7ef1cf7
Model has 8 quadratic constraints
Variable types: 0 continuous, 33 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [5e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 29 rows and 4 columns
Presolve time: 0.00s
Presolved: 30 rows, 53 columns, 74 nonzeros
Presolved model has 16 SOS constraint(s)
Variable types: 0 continuous, 53 integer (16 binary)
Found heuristic solution: objective 859000.00000
Found heuristic solution: objective 830000.00000
Found heuristic solution: objective 822000.00000

Root relaxation: objective 7.170501e+05,

In [ ]:
for v in m.getVars():

	if v.varName.startswith("x"):
		print("Produce {} products in month {}".format(v.x, v.varName[1:]))

print("--------------------------------")
print("Total cost: {}".format(m.objVal))

Produce 600.0 products in month [0]
Produce 0.0 products in month [1]
Produce 1600.0 products in month [2]
Produce 0.0 products in month [3]
Produce 1200.0 products in month [4]
Produce 1200.0 products in month [5]
Produce 1200.0 products in month [6]
Produce 1200.0 products in month [7]
--------------------------------
Total cost: 736000.0
